# **5) Stitching nets**

In [7]:
from pathlib import Path
import os, sys, platform, json, subprocess

ROOT = Path.cwd().resolve().parents[0]

if (ROOT / "src").exists():
    sys.path.insert(0, str(ROOT))

print("Project root:", ROOT)
print("Python:", sys.version)
print("OS:", platform.platform())

Project root: D:\IIT BBS\Job Resources\fiolabs\diagram-intel
Python: 3.11.13 | packaged by Anaconda, Inc. | (main, Jun  5 2025, 13:03:15) [MSC v.1929 64 bit (AMD64)]
OS: Windows-10-10.0.26100-SP0


In [8]:
from src.config.loader import load_cfg
from src.stitching.build_nets import stitch_page
from src.utils.io import read_json

cfg = load_cfg()
pdf_stem = Path(next(Path(cfg.paths.input_pdfs).glob("*.pdf"))).stem

In [3]:
G, nets_payload = stitch_page(cfg, pdf_stem, page=1)
nets_payload["count"], nets_payload["nets"][:3]

2025-08-27 11:51:11.276 | INFO     | src.graph.build_graph:build_graph_for_page:88 - [graph.build] nodes=4510 edges=5607
2025-08-27 11:51:11.659 | INFO     | src.stitching.build_nets:stitch_page:144 - [stitch] wiring diagram_Main Distribution Panel_Fio Labs page-1: nets=158 → D:\IIT BBS\Job Resources\fiolabs\diagram-intel\data\processed\nets\wiring diagram_Main Distribution Panel_Fio Labs\page-1.json


(158,
 [{'net_id': 0,
   'nodes': 4160,
   'components': 4,
   'ports': 2117,
   'junctions': 2039,
   'phase': 'L1/L2/L3/N',
   'voltage': None},
  {'net_id': 1,
   'nodes': 2,
   'components': 0,
   'ports': 0,
   'junctions': 2,
   'phase': None,
   'voltage': None},
  {'net_id': 2,
   'nodes': 2,
   'components': 0,
   'ports': 0,
   'junctions': 2,
   'phase': None,
   'voltage': None}])

In [4]:
from pathlib import Path, PurePath
graph_json = Path(cfg.paths.processed)/"graphs"/pdf_stem/"page-1.json"
read_json(graph_json)["nodes"][:3]

[{'id': 'comp:wiring diagram_Main Distribution Panel_Fio Labs:1:comp:1:5:610625',
  'kind': 'component',
  'comp_id': 'wiring diagram_Main Distribution Panel_Fio Labs:1:comp:1:5:610625',
  'bbox': [0.0, 0.0, 7984.0, 1894.0],
  'type': None,
  'confidence': 0.0,
  'labels_context': 'N 1 3 | N 2 4 | 4 6 | 20G | 55G | 57G | 70G | 122G | 124G | 121G | 125G | 142G | RCCB1 40A 100mA | Isolator2 40A | 1 3 | 127G | 140G | 141G | RCCB2 40A 100mA | ACCL | MODULE | EB | DG | LOAD | 107G | 110G | 111G | 1Ph ACCL | 105G | POS-1--Bypass | POS-2--ACCL | A1 | A2 | A3 | B1 | B2 | B3 | 115G | 120G | 114G | ACCL Bypass | MCB11 32A | Isolator1 40A | N 2 4 6 | 144G | 145G | 147G | 150G | RCCB3 25A 100mA | From | Inverter | Inverter Bypass | 2Pole, 3WAY | POS-1--Man EB | POS-2-Man DG | POS-3--Auto | To | TB1-4',
  'net_id': 0,
  'net_phase': 'L1/L2/L3/N',
  'net_voltage': None},
 {'id': 'comp:wiring diagram_Main Distribution Panel_Fio Labs:1:comp:5:4:438942',
  'kind': 'component',
  'comp_id': 'wiring diag

In [5]:
sum(1 for _,a in G.nodes(data=True) if a.get("net_id")==0 and a.get("kind")=="component"),
sum(1 for _,a in G.nodes(data=True) if a.get("net_id")==0 and a.get("kind")=="port"),
sum(1 for _,a in G.nodes(data=True) if a.get("net_id")==0 and a.get("kind")=="junction")

2039

In [6]:
(
  sum(1 for _,a in G.nodes(data=True) if a.get("net_id")==0 and a.get("kind")=="component"),
  sum(1 for _,a in G.nodes(data=True) if a.get("net_id")==0 and a.get("kind")=="port"),
  sum(1 for _,a in G.nodes(data=True) if a.get("net_id")==0 and a.get("kind")=="junction"),
)

(4, 2117, 2039)

In [9]:
from src.config.loader import load_cfg
from src.refine.autodiscover import discover_constraints_candidates
cfg = load_cfg()
pdf_stem = Path(next(Path(cfg.paths.input_pdfs).glob("*.pdf"))).stem
discover_constraints_candidates(cfg, pdf_stem)

2025-08-27 12:13:18.707 | INFO     | src.refine.autodiscover:discover_constraints_candidates:96 - [autodiscover] wrote suggestions → D:\IIT BBS\Job Resources\fiolabs\diagram-intel\configs\constraints\projects\suggested_wiring diagram_Main Distribution Panel_Fio Labs.yaml


{'path': 'D:\\IIT BBS\\Job Resources\\fiolabs\\diagram-intel\\configs\\constraints\\projects\\suggested_wiring diagram_Main Distribution Panel_Fio Labs.yaml',
 'sources_found': 30,
 'loads_found': 30,
 'device_clues': ['MCB',
  'TPN',
  'ACCL',
  'TB',
  'RCCB',
  'Isolator',
  'SPD',
  'MCCB',
  'CTS']}